In [50]:
!pip install imbalanced-learn
!pip install pandas geopandas


You should consider upgrading via the 'C:\Users\CHAKRAVARTHY\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'C:\Users\CHAKRAVARTHY\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


In [ ]:
import geopandas as gpd
import pandas as pd

file_path = 'C:/Users/CHAKRAVARTHY/Downloads/az_dhc_2020_t/az_dhc_2020_t.csv'
csv_data = pd.read_csv(file_path)

file_path1 = 'C:/Users/CHAKRAVARTHY/Downloads/az_vest_20/az_vest_20.shp'
shapefile = gpd.read_file(file_path1)


print(csv_data.shape)
print(shapefile.shape)
print(csv_data['COUNTY'].unique())
print(shapefile['CDE_COUNTY'].unique())
#print(shapefile['PCTNUM'].unique())
#print(shapefile['PRECINCTNA'].unique())

(1765, 90)
(1489, 9)
['Maricopa County' 'Gila County' 'La Paz County' 'Santa Cruz County'
 'Pinal County' 'Yavapai County' 'Yuma County' 'Coconino County'
 'Navajo County' 'Apache County' 'Cochise County' 'Graham County'
 'Greenlee County' 'Mohave County' 'Pima County']
['AP' 'CH' 'CN' 'GI' 'GM' 'LP' 'MO' 'PN' 'SC' 'YA' 'NA' 'YU' 'MC' 'PM'
 'GN']
['AP0002' 'AP0003' 'AP0005' ... 'YA0232' 'YA0222' 'YA0225']
['ALPINE' 'CANYON DE CHELLY' 'CHINLE' ... 'GHOST' 'BUG' 'IRON']


In [52]:
import pandas as pd


# Mapping dictionary
county_map = {
    'Maricopa County': 'MC',
    'Gila County': 'GI',
    'La Paz County': 'LP',
    'Santa Cruz County': 'SC',
    'Pinal County': 'PN',
    'Yavapai County': 'YA',
    'Yuma County': 'YU',
    'Coconino County': 'CO',
    'Navajo County': 'NA',
    'Apache County': 'AP',
    'Cochise County': 'CH',
    'Graham County': 'GM',
    'Greenlee County': 'GN',
    'Mohave County': 'MO',
    'Pima County': 'PM'
}

# Replace full county names with abbreviations
csv_data['CDE_COUNTY'] = csv_data['COUNTY'].map(county_map)

print(csv_data['CDE_COUNTY'].unique())


['MC' 'GI' 'LP' 'SC' 'PN' 'YA' 'YU' 'CO' 'NA' 'AP' 'CH' 'GM' 'GN' 'MO'
 'PM']


In [53]:
# Inspect the shapefile columns
print(shapefile.columns)

# Inspect the CSV columns
print(csv_data.columns)



Index(['CDE_COUNTY', 'PCTNUM', 'PRECINCTNA', 'G20PREDBID', 'G20PRERTRU',
       'G20PRELJOR', 'G20USSDKEL', 'G20USSRMCS', 'geometry'],
      dtype='object')
Index(['GEOID20', 'STATEFP20', 'STATE', 'COUNTYFP20', 'COUNTY', 'TOT_POP',
       'URBAN', 'RURAL', 'NOT_DEF', 'MED_AGE', 'MED_AGE_M', 'MED_AGE_F',
       'MALE', 'U5_M', '5_9_M', '10_14_M', '15_17_M', '18_19_M', '20_M',
       '21_M', '22_24_M', '25_29_M', '30_34_M', '35_39_M', '40_44_M',
       '45_49_M', '50_54_M', '55_59_M', '60_61_M', '62_64_M', '65_66_M',
       '67_69_M', '70_74_M', '75_79_M', '80_84_M', '85_O_M', 'FEMALE', 'U5_F',
       '5_9_F', '10_14_F', '15_17_F', '18_19_F', '20_F', '21_F', '22_24_F',
       '25_29_F', '30_34_F', '35_39_F', '40_44_F', '45_49_F', '50_54_F',
       '55_59_F', '60_61_F', '62_64_F', '65_66_F', '67_69_F', '70_74_F',
       '75_79_F', '80_84_F', '85_O_F', 'IN_HH_POP', 'HH_IN_HH', 'M_HH',
       'M_ALONE', 'M_N_ALONE', 'F_HH', 'F_ALONE', 'F_N_ALONE', 'O_SEX_MAR',
       'S_SEX_MAR', 'O_SEX_UMA

In [ ]:
import numpy as np

# Create a function to calculate the winner based on the majority of votes
def calculate_winner(row):
    # Votes for each party
    republican_votes = row['G20PRERTRU'] + row['G20USSRMCS'] 
    democrat_votes = row['G20PREDBID'] + row['G20USSDKEL'] 
    other_votes = row['G20PRELJOR']                       

    # Determine the winner based on vote comparison
    if republican_votes > democrat_votes and republican_votes > other_votes:
        return 'R'  # Republican Party
    elif democrat_votes > republican_votes and democrat_votes > other_votes:
        return 'D'  # Democratic Party
    else:
        return 'Other'  # Libertarian or others

# Apply the function to create a new column 'Winner'
shapefile['Winner'] = shapefile.apply(calculate_winner, axis=1)

# Check the updated dataframe
print(shapefile[['G20PRERTRU', 'G20PREDBID', 'G20PRELJOR', 'G20USSDKEL', 'G20USSRMCS', 'Winner']].head())

print(shapefile['Winner'].unique())


   G20PRERTRU  G20PREDBID  G20PRELJOR  G20USSDKEL  G20USSRMCS Winner
0         283          94           5          98         284      R
1         273        1982          30        1995         290      D
2         148         989          20         994         157      D
3        1486         333          22         381        1422      R
4          87         748           5         763          85      D
['R' 'D' 'Other']


In [55]:
# Group by the 'CDE_COUNTY' column (the county column)
grouped_counties = shapefile.groupby('CDE_COUNTY')

# Initialize empty dictionaries to store the sum of votes by county
republican_votes_by_county = {}
democrat_votes_by_county = {}
other_votes_by_county = {}
winner_by_county = {}

# Iterate through each group (county) and calculate the vote totals
for county, group in grouped_counties:
    sum_republican_votes = group['G20PRERTRU'].sum() + group['G20USSRMCS'].sum()
    sum_democrat_votes = group['G20PREDBID'].sum() + group['G20USSDKEL'].sum()
    sum_other_votes = group['G20PRELJOR'].sum()
    
    # Store the vote totals in dictionaries, using the county as the key
    republican_votes_by_county[county] = sum_republican_votes
    democrat_votes_by_county[county] = sum_democrat_votes
    other_votes_by_county[county] = sum_other_votes

    # Determine the winner based on highest vote count
    if sum_republican_votes > sum_democrat_votes and sum_republican_votes > sum_other_votes:
        winner_by_county[county] = 'Republican'
    elif sum_democrat_votes > sum_republican_votes and sum_democrat_votes > sum_other_votes:
        winner_by_county[county] = 'Democrat'
    else:
        winner_by_county[county] = 'Other'

# Convert the dictionaries into DataFrames for easy viewing or further analysis
votes_summary = pd.DataFrame({
    'Republican_Votes': republican_votes_by_county,
    'Democrat_Votes': democrat_votes_by_county,
    'Other_Votes': other_votes_by_county,
    'Winner': winner_by_county
})

# Print the summary of votes and winner by county
print(votes_summary)


    Republican_Votes  Democrat_Votes  Other_Votes      Winner
AP             22494           47343          437    Democrat
CH             70771           48575         1153  Republican
CN             54307           90259         1522    Democrat
GI             36266           18591          342  Republican
GM             21052            8540          212  Republican
GN              4658            2585           70  Republican
LP              9964            4728           93  Republican
MC           1979868         2105170        31705    Democrat
MO            153894           52245         1302  Republican
NA             54609           47779          727  Republican
PM            415075          614123         7658    Democrat
PN            211125          154220         2791  Republican
SC             11879           26827          224    Democrat
YA            181707          101454         2092  Republican
YU             71678           66171         1137  Republican


In [61]:
# Mapping dictionary for counties and their winners
county_winner_map = {
    'AP': 'Democrat',
    'CH': 'Republican',
    'CO': 'Democrat',
    'GI': 'Republican',
    'GM': 'Republican',
    'GN': 'Republican',
    'LP': 'Republican',
    'MC': 'Democrat',
    'MO': 'Republican',
    'NA': 'Republican',
    'PM': 'Democrat',
    'PN': 'Republican',
    'SC': 'Democrat',
    'YA': 'Republican',
    'YU': 'Republican'
}

# Replace full county names with abbreviations
csv_data['PRES_WIN'] = csv_data['CDE_COUNTY'].map(county_winner_map)

print(csv_data['PRES_WIN'].unique())


['Democrat' 'Republican']


In [62]:
# Display rows where 'PRES_WIN' has NaN values
nan_entries = csv_data[csv_data['PRES_WIN'].isna()]

print(nan_entries)

print(nan_entries['CDE_COUNTY'].unique())


Empty DataFrame
Columns: [GEOID20, STATEFP20, STATE, COUNTYFP20, COUNTY, TOT_POP, URBAN, RURAL, NOT_DEF, MED_AGE, MED_AGE_M, MED_AGE_F, MALE, U5_M, 5_9_M, 10_14_M, 15_17_M, 18_19_M, 20_M, 21_M, 22_24_M, 25_29_M, 30_34_M, 35_39_M, 40_44_M, 45_49_M, 50_54_M, 55_59_M, 60_61_M, 62_64_M, 65_66_M, 67_69_M, 70_74_M, 75_79_M, 80_84_M, 85_O_M, FEMALE, U5_F, 5_9_F, 10_14_F, 15_17_F, 18_19_F, 20_F, 21_F, 22_24_F, 25_29_F, 30_34_F, 35_39_F, 40_44_F, 45_49_F, 50_54_F, 55_59_F, 60_61_F, 62_64_F, 65_66_F, 67_69_F, 70_74_F, 75_79_F, 80_84_F, 85_O_F, IN_HH_POP, HH_IN_HH, M_HH, M_ALONE, M_N_ALONE, F_HH, F_ALONE, F_N_ALONE, O_SEX_MAR, S_SEX_MAR, O_SEX_UMAR, S_SEX_UMAR, BIO_CHLD, BIO_U18, ADPT_CHLD, ADPT_U18, STEP_CHLD, STEP_U18, GRAND_CHLD, GRAND_U18, SIBLING, PARENT, PAR_INLAW, CHLD_INLAW, OTH_REL, FSTR_CHLD, OTH_NONREL, GRP_QUART, GRP_INST, GRP_N_INST, CDE_COUNTY, PRES_WIN]
Index: []

[0 rows x 92 columns]
[]


In [65]:
print(csv_data)


         GEOID20  STATEFP20    STATE  COUNTYFP20           COUNTY  TOT_POP  \
0     4013422625          4  Arizona        4013  Maricopa County     4405   
1     4013422626          4  Arizona        4013  Maricopa County     2719   
2     4013422627          4  Arizona        4013  Maricopa County     3576   
3     4013422628          4  Arizona        4013  Maricopa County     4065   
4     4013422629          4  Arizona        4013  Maricopa County     3577   
...          ...        ...      ...         ...              ...      ...   
1760  4019004313          4  Arizona        4019      Pima County     4529   
1761  4019004316          4  Arizona        4019      Pima County     3106   
1762  4019004317          4  Arizona        4019      Pima County     3160   
1763  4019004320          4  Arizona        4019      Pima County     2527   
1764  4019004321          4  Arizona        4019      Pima County     4569   

      URBAN  RURAL  NOT_DEF  MED_AGE  ...  PAR_INLAW  CHLD_INLA

In [ ]:
# Filter the CSV data for specific counties
filtered_csv_data = csv_data[csv_data['CDE_COUNTY'].isin(['CO', 'MC', 'PN', 'PM', 'SC', 'YU'])]
# Filter the CSV data for records that are not from the specified counties
filtered_csv_data1 = csv_data[~csv_data['CDE_COUNTY'].isin(['CO', 'MC', 'PN', 'PM', 'SC', 'YU'])]


# Display the filtered data
print(filtered_csv_data)
print(filtered_csv_data['CDE_COUNTY'].unique())
print(filtered_csv_data1)
print(filtered_csv_data1['CDE_COUNTY'].unique())


         GEOID20  STATEFP20    STATE  COUNTYFP20         COUNTY  TOT_POP  \
1009  4007000100          4  Arizona        4007    Gila County     2908   
1010  4007000200          4  Arizona        4007    Gila County     2132   
1011  4007000301          4  Arizona        4007    Gila County     5775   
1012  4007000302          4  Arizona        4007    Gila County     3042   
1013  4007000400          4  Arizona        4007    Gila County     5307   
...          ...        ...      ...         ...            ...      ...   
1490  4015953406          4  Arizona        4015  Mohave County     2631   
1491  4015953602          4  Arizona        4015  Mohave County     2718   
1492  4015953603          4  Arizona        4015  Mohave County     4292   
1493  4015953604          4  Arizona        4015  Mohave County     6675   
1494  4015953800          4  Arizona        4015  Mohave County     6659   

      URBAN  RURAL  NOT_DEF  MED_AGE  ...  PAR_INLAW  CHLD_INLAW  OTH_REL  \
1009      

In [73]:
print(filtered_csv_data.columns)
filtered_csv_data.dtypes

Index(['GEOID20', 'STATEFP20', 'STATE', 'COUNTYFP20', 'COUNTY', 'TOT_POP',
       'URBAN', 'RURAL', 'NOT_DEF', 'MED_AGE', 'MED_AGE_M', 'MED_AGE_F',
       'MALE', 'U5_M', '5_9_M', '10_14_M', '15_17_M', '18_19_M', '20_M',
       '21_M', '22_24_M', '25_29_M', '30_34_M', '35_39_M', '40_44_M',
       '45_49_M', '50_54_M', '55_59_M', '60_61_M', '62_64_M', '65_66_M',
       '67_69_M', '70_74_M', '75_79_M', '80_84_M', '85_O_M', 'FEMALE', 'U5_F',
       '5_9_F', '10_14_F', '15_17_F', '18_19_F', '20_F', '21_F', '22_24_F',
       '25_29_F', '30_34_F', '35_39_F', '40_44_F', '45_49_F', '50_54_F',
       '55_59_F', '60_61_F', '62_64_F', '65_66_F', '67_69_F', '70_74_F',
       '75_79_F', '80_84_F', '85_O_F', 'IN_HH_POP', 'HH_IN_HH', 'M_HH',
       'M_ALONE', 'M_N_ALONE', 'F_HH', 'F_ALONE', 'F_N_ALONE', 'O_SEX_MAR',
       'S_SEX_MAR', 'O_SEX_UMAR', 'S_SEX_UMAR', 'BIO_CHLD', 'BIO_U18',
       'ADPT_CHLD', 'ADPT_U18', 'STEP_CHLD', 'STEP_U18', 'GRAND_CHLD',
       'GRAND_U18', 'SIBLING', 'PARENT', 'PAR

GEOID20        int64
STATEFP20      int64
STATE         object
COUNTYFP20     int64
COUNTY        object
               ...  
GRP_QUART      int64
GRP_INST       int64
GRP_N_INST     int64
CDE_COUNTY    object
PRES_WIN      object
Length: 92, dtype: object

In [74]:
# Assuming your DataFrame is named 'csv_data'
filtered_csv_data = filtered_csv_data.drop_duplicates()


In [75]:
print(filtered_csv_data)

         GEOID20  STATEFP20    STATE  COUNTYFP20           COUNTY  TOT_POP  \
0     4013422625          4  Arizona        4013  Maricopa County     4405   
1     4013422626          4  Arizona        4013  Maricopa County     2719   
2     4013422627          4  Arizona        4013  Maricopa County     3576   
3     4013422628          4  Arizona        4013  Maricopa County     4065   
4     4013422629          4  Arizona        4013  Maricopa County     3577   
...          ...        ...      ...         ...              ...      ...   
1760  4019004313          4  Arizona        4019      Pima County     4529   
1761  4019004316          4  Arizona        4019      Pima County     3106   
1762  4019004317          4  Arizona        4019      Pima County     3160   
1763  4019004320          4  Arizona        4019      Pima County     2527   
1764  4019004321          4  Arizona        4019      Pima County     4569   

      URBAN  RURAL  NOT_DEF  MED_AGE  ...  PAR_INLAW  CHLD_INLA

In [76]:
print("Null values count in each column:")
print(filtered_csv_data.isnull().sum())

Null values count in each column:
GEOID20       0
STATEFP20     0
STATE         0
COUNTYFP20    0
COUNTY        0
             ..
GRP_QUART     0
GRP_INST      0
GRP_N_INST    0
CDE_COUNTY    0
PRES_WIN      0
Length: 92, dtype: int64


In [78]:
y = filtered_csv_data[['PRES_WIN']]
X = filtered_csv_data.drop(columns=['PRES_WIN'])
print("Features (X):")
print(X)
print("\nTarget (y):")
print(y)
X_train, X_test, y_train, y_test = train_test_split(X,
y,
test_size=.2,
stratify=y,
random_state=42)

Features (X):
         GEOID20  STATEFP20    STATE  COUNTYFP20           COUNTY  TOT_POP  \
0     4013422625          4  Arizona        4013  Maricopa County     4405   
1     4013422626          4  Arizona        4013  Maricopa County     2719   
2     4013422627          4  Arizona        4013  Maricopa County     3576   
3     4013422628          4  Arizona        4013  Maricopa County     4065   
4     4013422629          4  Arizona        4013  Maricopa County     3577   
...          ...        ...      ...         ...              ...      ...   
1760  4019004313          4  Arizona        4019      Pima County     4529   
1761  4019004316          4  Arizona        4019      Pima County     3106   
1762  4019004317          4  Arizona        4019      Pima County     3160   
1763  4019004320          4  Arizona        4019      Pima County     2527   
1764  4019004321          4  Arizona        4019      Pima County     4569   

      URBAN  RURAL  NOT_DEF  MED_AGE  ...  PARENT

In [79]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score


# Preprocessing for numerical and categorical features
numeric_features = X.select_dtypes(include=['float64', 'int64']).columns
categorical_features = X.select_dtypes(include=['object']).columns

# Preprocessing pipeline: scale numeric features, and one-hot encode categorical features
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numeric_features),
        ('cat', OneHotEncoder(), categorical_features)
    ])

# Define the model (RandomForestClassifier for simplicity)
model = RandomForestClassifier(random_state=42)

# Create a pipeline: first preprocess the data, then fit the model
pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', model)
])

# Train the model
pipeline.fit(X_train, y_train.values.ravel())

# Predict on the test set
y_pred = pipeline.predict(X_test)

# Evaluate the model
print("Accuracy:", accuracy_score(y_test, y_pred))
print("\nClassification Report:")
print(classification_report(y_test, y_pred))


Accuracy: 1.0

Classification Report:
              precision    recall  f1-score   support

    Democrat       1.00      1.00      1.00       267
  Republican       1.00      1.00      1.00        32

    accuracy                           1.00       299
   macro avg       1.00      1.00      1.00       299
weighted avg       1.00      1.00      1.00       299



In [ ]:


count = y_train['PRES_WIN'].value_counts()

# Determine the winner
winner = count.idxmax() 
winner_count = count.max()  

# Display the overall winner and the number of precincts won
print(f"Overall Winner: {winner}")
print(f"Number of Precincts Won: {winner_count}")


total_data = len(filtered_csv_data)
winner_percentage = (winner_count / total_data) * 100

print(f"Winner Percentage: {winner_percentage:.2f}%")


Overall Winner: Democrat
Number of Precincts Won: 1065
Winner Percentage: 71.29%


In [ ]:

predicted_output = pd.Series(y_pred, name='predicted_PRES_WIN')


predicted_count = predicted_output.value_counts()

# Determine the predicted winner
predicted_winner = predicted_count.idxmax() 
predicted_winner_count = predicted_count.max()  

total_ = len(y_pred)
predicted_winner_percentage = (predicted_winner_count / total_) * 100

# Display the predicted
print(f"Predicted Winner: {predicted_winner}")
print(f"Predicted Number of Precincts Won: {predicted_winner_count}")
print(f"Predicted Winner Percentage: {predicted_winner_percentage:.2f}%")


Predicted Winner: Democrat
Predicted Number of Precincts Won: 267
Predicted Winner Percentage: 89.30%
